In [48]:
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from 

In [2]:
X = np.load('data/games_ar.npy', allow_pickle=True)
y = np.load('data/winner.npy', allow_pickle=True)

In [56]:
def train(net, X, y, EPOCHS=8, BATCH_SIZE=100):
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(X), BATCH_SIZE)):
            batch_X = X[i:i+BATCH_SIZE].cuda()
            batch_y = y[i:i+BATCH_SIZE].cuda()
            
            net.zero_grad()
            output = net(batch_X)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()
        
        print(loss)

In [4]:
def test(net, X, y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        output = net(X.cuda())
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
    net.train()
    return round(correct/total,3)

In [38]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.1)

train_X = torch.tensor(train_X, dtype=torch.float)
train_y = torch.tensor(train_y, dtype=torch.long)
test_X = torch.tensor(test_X, dtype=torch.float)
test_y = torch.tensor(test_y, dtype=torch.long)

In [60]:
features = [
    nn.Linear(train_X.shape[1], 128),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(128, 2)]

net = nn.Sequential(*features).cuda()

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
train(net, train_X, train_y)
print(f"Train accuracy: {test(net, train_X, train_y)}")
print(f"Test accuracy: {test(net, test_X, test_y)}")

  0%|                                                                                           | 0/14 [00:00<?, ?it/s]

tensor(0.7028, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6684, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6086, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.5230, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3824, device='cuda:0', grad_fn=<NllLossBackward>)


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 494.35it/s]


tensor(0.2605, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1896, device='cuda:0', grad_fn=<NllLossBackward>)
Train accuracy: 0.875
Test accuracy: 0.591


In [8]:
# overfitted but 60% over 150 games is significantly better than guessing
test_X.shape[0]

149

In [62]:
def save_model(path):
    torch.save(net.state_dict(), f'models/{path}')
    print(f"Model saved to 'models/{path}'!")

In [10]:
def load_model(path):
    model = nn.Sequential(*features).cuda()
    model.load_state_dict(torch.load(f'models/{path}'))
    model.eval()
    return model

In [63]:
# save_model('sixty.pth')

Model saved to 'models/sixty.pth'!


In [82]:
def parse_champs(game):
    import pickle
    import numpy as np
    with open('data/champ_dict.pkl', 'rb') as f:
        champ_dict = pickle.load(f)
        
    picked = []
    picked.append([champ_dict[champ] for champ in game])
    picked = picked[0]
    
    five_hot1 = np.zeros((len(champ_dict),), dtype=int)
    for j in picked[0:5]:
        five_hot1[j] = 1
    five_hot2 = np.zeros((len(champ_dict),), dtype=int)
    for k in picked[5:10]:
        five_hot2[k] = 1
    return np.concatenate((five_hot1, five_hot2))

In [89]:
def custom_test(net, game):
    ten_hot = parse_champs(game)
    custom_X = torch.tensor(ten_hot, dtype=torch.float)
    net.eval()
    with torch.no_grad():
        output = net(custom_X.to(device='cuda'))
    net.train()
    
    return torch.argmax(output)

In [91]:
# TODO: fix cuda error
custom_test(net, ['Renekton', 'Lee Sin', 'Syndra', 'Draven', 'Blitzcrank', 'Ornn', 'Nidalee', 'Orianna', 'Ashe', 'Sett'])

RuntimeError: CUDA error: an illegal memory access was encountered